# <u><center> Project 5: Part 3 Core
* Authored By: Eric N. Valdez
* Date: 03-17-2024

# <u>Imports:

# <u>Custome Functions:

# <u>Task:
* Build a small neural Network (with only 1 hidden layer)
   * Ensure the Final layer is appropriate for your task (number of units and activation function)
   * Fit it for 50 epochs
       *  Use Early Stopping callbacks
           * Start with patience = 5 monitoring val_accuracy.
       * Use a validation_split of 2
       * Save the history and visualize it.  
    * Evaluate the model using sklearn evaluation metrics:
        * Confusion Matrix
        * Classification Report
        * You are Welcome to use the custom  evaluation functions  

* Tune at least 3 parameters with Keras tuner:
    * Include a dropout layer and adjust the dropout rate
    * Number of units
    * Optimizer
    * Learning rates 
* Evaluate your best model on unseen test data